This code makes an exapmle of an address which has a negtive amount

In [16]:
import sqlite3
import json

# Connect to the SQLite database
conn = sqlite3.connect('iotaDB.db')
cursor = conn.cursor()

# Specific address to analyze
specific_address = '26d651d5fa1491ef2024cece03124cd1f16a8ac7377f0be1f8576d5140f19489'

# Query the database for transactions involving the specific address
query = f"""
SELECT transaction_id, input_addresses_x, input_amounts_x, output_addresses_y, output_amounts_y 
FROM Transactions 
WHERE input_addresses_x LIKE '%{specific_address}%' OR output_addresses_y LIKE '%{specific_address}%'
"""
cursor.execute(query)
rows = cursor.fetchall()

# Function to print and update balance based on transaction data
def update_and_print_balance(address, amounts, transaction_id, is_input=True):
    global address_balance
    for addr, amt in zip(address, amounts):
        if addr == specific_address:
            if is_input:
                address_balance -= amt
                print(f"Transaction ID: {transaction_id} | Input: -{amt} | New balance: {address_balance}")
            else:
                address_balance += amt
                print(f"Transaction ID: {transaction_id} | Output: +{amt} | New balance: {address_balance}")

# Initialize balance for the address
address_balance = 0

# Iterate over each row to parse JSON and accumulate amounts for the specific address
for transaction_id, input_addresses_x, input_amounts_x, output_addresses_y, output_amounts_y in rows:
    # Parse the JSON data
    input_addresses = json.loads(input_addresses_x)
    input_amounts = json.loads(input_amounts_x)
    output_addresses = json.loads(output_addresses_y)
    output_amounts = json.loads(output_amounts_y)

    # Check and update balances for input and output addresses, and print transaction details including the transaction ID
    update_and_print_balance(input_addresses, input_amounts, transaction_id, is_input=True)
    update_and_print_balance(output_addresses, output_amounts, transaction_id, is_input=False)


conn.close()

# Print the final balance for the specific address
print(f'Final balance for address {specific_address}: {address_balance}')


Transaction ID: b4038ac05b09c9bff9ea5ae9bc554cf023b125218cd132bc832a910c0936fb02 | Input: -631980000.0 | New balance: -631980000.0
Transaction ID: 0c6d57dd74e289d097a5a4da36f4c44b184e113eef1b85b20f96efa15de6509b | Output: +631979494.0 | New balance: -506.0
Final balance for address 26d651d5fa1491ef2024cece03124cd1f16a8ac7377f0be1f8576d5140f19489: -506.0
